In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.optimizers import Adam

In [4]:
# Load dataset
df = pd.read_csv('data/phishing_email.csv')  # Adjust the file path as necessary

# Preprocess the data
X = df['text_combined'].values  # Assuming the first column is named 'text'
y = df['label'].values  # Assuming the second column is named 'label'

# Tokenization and Padding
tokenizer = Tokenizer(num_words=5000, lower=True, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=100)  # Adjust maxlen as necessary

# Encoding labels
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f'Validation Accuracy: {accuracy*100:.2f}%')

Epoch 1/5


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1032/1032 ━━━━━━━━━━━━━━━━━━━━ 49s 46ms/step - accuracy: 0.8809 - loss: 0.2522 - val_accuracy: 0.9790 - val_loss: 0.0625
Epoch 2/5
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 48s 47ms/step - accuracy: 0.9802 - loss: 0.0620 - val_accuracy: 0.9803 - val_loss: 0.0602
Epoch 3/5
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 46s 45ms/step - accuracy: 0.9854 - loss: 0.0460 - val_accuracy: 0.9792 - val_loss: 0.0607
Epoch 4/5
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 47s 46ms/step - accuracy: 0.9887 - loss: 0.0340 - val_accuracy: 0.9792 - val_loss: 0.0656
Epoch 5/5
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 47s 45ms/step - accuracy: 0.9914 - loss: 0.0262 - val_accuracy: 0.9801 - val_loss: 0.0602
516/516 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9815 - loss: 0.0593
Validation Accuracy: 98.01%


In [ ]:
model.save("phishing_email_detector.keras")